In [1]:
# 필요한 라이브러리 설치
!pip install holidays

# 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import holidays

# 데이터 파일 경로
file_paths = [
    r"C:\Users\james\DATA_LAB\KTX_Forecasting\data\(간선)수송-운행일-주운행(201501-202305).xlsx",
    r"C:\Users\james\DATA_LAB\KTX_Forecasting\data\(간선)수송-운행일-주운행(202305-202403).xlsx",
    r"C:\Users\james\DATA_LAB\KTX_Forecasting\data\(간선)시종착역별 열차운행(201501-202305).xlsx",
    r"C:\Users\james\DATA_LAB\KTX_Forecasting\data\(간선)시종착역별 열차운행(202305-202403).xlsx"
]

# 데이터 로드 (지정된 행을 열 이름으로 사용)
data_frames = [
    pd.read_excel(file_paths[0], header=5),
    pd.read_excel(file_paths[1], header=5),
    pd.read_excel(file_paths[2], header=8),
    pd.read_excel(file_paths[3], header=8)
]

# 데이터 결합
df_demand = pd.concat([data_frames[0], data_frames[1]], ignore_index=True)
df_info = pd.concat([data_frames[2], data_frames[3]], ignore_index=True)

# 불필요한 열 삭제
df_demand.drop(columns=['Unnamed: 1', '운행년도', '운행년월', '운행요일구분', '역무열차종', '메트릭'], inplace=True)
df_info.drop(columns=['상행하행구분', '역무열차종', '운행요일구분', '메트릭'], inplace=True)

# 주운행선 필터링
df_demand = df_demand[df_demand['주운행선'].isin(['경부선', '경전선', '동해선', '호남선', '전라선'])]
df_info = df_info[df_info['주운행선'].isin(['경부선', '경전선', '동해선', '호남선', '전라선'])]
df_demand = df_demand.reset_index(drop=True)
df_info = df_info.reset_index(drop=True)

# 날짜 형식 통일
df_demand['운행일자'] = pd.to_datetime(df_demand['운행일자'], format='%Y년 %m월 %d일')
df_info['운행일자'] = pd.to_datetime(df_info['운행일자'], format='%Y년 %m월 %d일')

# 결측치 처리
df_demand = df_demand.dropna()
df_info = df_info.dropna()

# 변수 생성
df_demand['1인당수입율'] = df_demand['승차수입금액'] / df_demand['승차인원수']
df_demand['공급대비승차율'] = df_demand['승차인원수'] / df_demand['공급좌석합계수']
df_demand['운행대비고객이동'] = df_demand['좌석거리'] / df_demand['승차연인거리']

# 요일 변수 생성
df_demand['요일'] = df_demand['운행일자'].dt.weekday
df_info['요일'] = df_info['운행일자'].dt.weekday
weekday_list = ['월', '화', '수', '목', '금', '토', '일']
df_demand['요일'] = df_demand['요일'].apply(lambda x: weekday_list[x])
df_info['요일'] = df_info['요일'].apply(lambda x: weekday_list[x])

# 주말/주중 변수 생성 (금요일 포함)
df_demand['주말여부'] = df_demand['요일'].apply(lambda x: '주말' if x in ['금', '토', '일'] else '주중')
df_info['주말여부'] = df_info['요일'].apply(lambda x: '주말' if x in ['금', '토', '일'] else '주중')

# 공휴일 변수 생성
kr_holidays = holidays.KR()

def is_holiday(date):
    return date in kr_holidays

df_demand['공휴일여부'] = df_demand['운행일자'].apply(lambda x: '공휴일' if is_holiday(x) else '평일')

# 명절 날짜 설정
seollal_dates = pd.to_datetime(['2015-02-19', '2016-02-08', '2017-01-28', '2018-02-16', '2019-02-05', '2020-01-25', '2021-02-12', '2022-02-01', '2023-01-22', '2024-02-10'])
chuseok_dates = pd.to_datetime(['2015-09-27', '2016-09-15', '2017-10-04', '2018-09-24', '2019-09-13', '2020-10-01', '2021-09-21', '2022-09-10', '2023-09-29', '2024-09-17'])

# 명절 연휴 포함 (설날, 설날 전날, 설날 다음날 / 추석, 추석 전날, 추석 다음날)
seollal_holidays = seollal_dates.union(seollal_dates - pd.Timedelta(days=1)).union(seollal_dates + pd.Timedelta(days=1))
chuseok_holidays = chuseok_dates.union(chuseok_dates - pd.Timedelta(days=1)).union(chuseok_dates + pd.Timedelta(days=1))

# 명절 변수 생성
df_demand['명절여부'] = df_demand['운행일자'].apply(lambda x: '명절' if x in seollal_holidays or x in chuseok_holidays else '비명절')

# 일자별 데이터를 기반으로 월별 변수 계산
all_dates = pd.date_range(start=df_demand['운행일자'].min(), end=df_demand['운행일자'].max(), freq='D')
date_info = pd.DataFrame(all_dates, columns=['운행일자'])

date_info['요일'] = date_info['운행일자'].dt.weekday
date_info['주말여부'] = date_info['요일'].apply(lambda x: '주말' if x in [4, 5, 6] else '주중')  # 금, 토, 일은 주말로 설정
date_info['공휴일여부'] = date_info['운행일자'].apply(lambda x: '공휴일' if is_holiday(x) else '평일')
date_info['명절여부'] = date_info['운행일자'].apply(lambda x: '명절' if x in seollal_holidays or x in chuseok_holidays else '비명절')
date_info['운행년월'] = date_info['운행일자'].dt.to_period('M')

monthly_info = date_info.groupby('운행년월').agg({
    '주말여부': lambda x: (x == '주말').sum(),
    '공휴일여부': lambda x: (x == '공휴일').sum(),
    '명절여부': lambda x: (x == '명절').sum(),
    '요일': 'count'  # 총 일수 계산
}).rename(columns={'주말여부': '주말수', '공휴일여부': '공휴일수', '명절여부': '명절수', '요일': '총일수'}).reset_index()

# 월별 집계
df_demand['운행년월'] = df_demand['운행일자'].dt.to_period('M')
df_info['운행년월'] = df_info['운행일자'].dt.to_period('M')

df_demand_monthly = df_demand.groupby(['주운행선', '운행년월']).agg({
    '공급차량수': 'sum',
    '공급좌석합계수': 'sum',
    '승차수입금액': 'sum',
    '승차인원수': 'sum',
    '승차연인거리': 'sum',
    '좌석거리': 'sum',
    '1인당수입율': 'mean',
    '공급대비승차율': 'mean',
    '운행대비고객이동': 'mean'
}).reset_index()

df_info_monthly = df_info.groupby(['주운행선', '운행년월']).sum().reset_index()

# 월별 데이터를 하나의 데이터프레임으로 결합
df_ktx = pd.merge(df_demand_monthly, df_info_monthly, on=['주운행선', '운행년월'], how='inner')

# 월별 변수 데이터와 결합
df_ktx = pd.merge(df_ktx, monthly_info, on='운행년월', how='left')

# '평일수' 계산
df_ktx['평일수'] = df_ktx['총일수'] - df_ktx['주말수']

# 불필요한 열 삭제
df_ktx.drop(columns=['총일수'], inplace=True)

# 결합된 데이터 확인
print("Combined Monthly DataFrame with additional variables:")
print(df_ktx.head())



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
C:\Users\james\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Combined Monthly DataFrame with additional variables:
  주운행선     운행년월  공급차량수  공급좌석합계수        승차수입금액    승차인원수       승차연인거리  \
0  경부선  2015-01  86203  4690099  148156008100  6462183  12887623030   
1  경부선  2015-02  78266  4275701  141350620300  6069139  12230649006   
2  경부선  2015-03  85116  4643846  139089651200  6565945  12032460501   
3  경부선  2015-04  81857  4467804  138289030400  6416662  11984070035   
4  경부선  2015-05  85949  4687943  155654821600  7082084  13565809400   

          좌석거리        1인당수입율   공급대비승차율  운행대비고객이동    공급좌석수    설비좌석수  열차운행횟수  \
0  18920136968  16791.258834  1.608243  1.596857  3543359  3565309    4035   
1  17328292972  17046.513104  1.696811  1.528621  3245150  3253174    3700   
2  18723639573  15629.282344  1.693707  1.754588  3508927  3518689    3999   
3  17902864800  17394.848015       NaN  1.772836  3397957  3409204    3810   
4  18795445014  17487.339237       NaN       NaN  3592912  3599102    4022   

   주말수  공휴일수  명절수  평일수  
0   14     1    0   17  


In [4]:
# 파일 경로
cpi_file_path = r"C:\Users\james\DATA_LAB\KTX_Forecasting\data\korea_cpi_data.xlsx"

# CPI 데이터 불러오기
df_cpi = pd.read_excel(cpi_file_path)

date_column = '발표일'  
actual_column = '실제'  
forecast_column = '예측' 

# '적용일' 변수 생성 (발표일의 한 달 전 날짜)
df_cpi['적용일'] = (df_cpi[date_column] - pd.DateOffset(months=1)).dt.to_period('M')


# '예상치 부합여부' 변수 생성 (부합이면 0, 예상치 하회면 -1, 예상치 상회면 1)
def determine_forecast_match(actual, forecast):
    if actual == forecast:
        return 0
    elif actual < forecast:
        return -1
    else:
        return 1

df_cpi['예상치 부합여부'] = df_cpi.apply(lambda row: determine_forecast_match(row[actual_column], row[forecast_column]), axis=1)

# CPI 데이터 확인
print("CHECK CPI DATA:")
print(df_cpi.head())

CHECK CPI DATA:
         발표일        시간     실제     예측     이전      적용일  예상치 부합여부
0 2024-04-02  08:00:00  0.031  0.031  0.031  2024-03         0
1 2024-03-06  08:00:00  0.031  0.029  0.028  2024-02         1
2 2024-02-02  08:00:00  0.028  0.029  0.032  2024-01        -1
3 2023-12-29  08:00:00  0.032  0.033  0.033  2023-11        -1
4 2023-12-05  08:00:00  0.033  0.037  0.038  2023-11        -1
